## 0)import libraries 

In [1]:
import os
import pickle

import dask.array as da
import numpy as np
import pandas as pd
import xarray as xr

from PyStemmusScope import variable_conversion as vc
from sklearn.preprocessing import OneHotEncoder
from dask.distributed import Client, LocalCluster
from dask_jobqueue import SLURMCluster

## 1)settings for Dask

In [2]:
# Set the MALLOC_TRIM_THRESHOLD_ environment variable
os.environ['MALLOC_TRIM_THRESHOLD_'] = '0'

In [3]:
# cluster = LocalCluster(n_workers=2, threads_per_worker=1)

Setup a Dask cluster on 6 * 16 = 96 cores (6 * 4 = 24 workers with 4 threads each) and 6 * 120 GB = 720 GB memory in total ('fat' nodes on Snellius):

In [4]:
cluster = SLURMCluster(
    name='dask-worker',
    cores=16,
    processes=4,
    queue='fat',
    memory='120GiB',
    local_directory='$TMPDIR',
    walltime='5:00:00'
)
cluster.scale(jobs=6)

In [6]:
client = Client(cluster)
client

<Client: 'tcp://145.136.57.179:35555' processes=20 threads=80, memory=600.00 GiB>

## 2)define working path, load trained model, define functions

In [7]:
year = 2015

In [8]:
ROOT_DIR = '/gpfs/work2/0/ttse0619'
DATA_DIR = f'{ROOT_DIR}/francesco/Projects/EcoExtreML/Data/1input_data/{year}global'
ERA5_PATH = f'{DATA_DIR}/era5land/era5land.zarr'
LAI_PATH = f'{DATA_DIR}/lai/lai.zarr'
LANDCOVER_PATH = f'{DATA_DIR}/igbp/landcover.zarr'
IGBP_CLASS_PATH = f'{ROOT_DIR}/qianqian/global_data_Qianqian/1input_data/IGBP11unique.csv'
IGBP_TABLE_PATH = f'{ROOT_DIR}/qianqian/global_data_Qianqian/1input_data/{year}global/igbp/lccs_to_igbp_table.csv' 

MODEL_PATH = f'{ROOT_DIR}/qianqian/global_data_Qianqian/3RF_train/RFLEH-LAI-hc-CO2-SSM-Vcmo-IGBP_multi7_1core_snellius0629.pkl'

LEH_PATH = f'{ROOT_DIR}/francesco/Projects/EcoExtreML/Data/5output_data/{year}fluxes.zarr'

In [9]:
# function for loading the trained model
def load_model(path):
    # load trained RF model, better use not parallel model with Dask
    with open(path, 'rb') as f:
        rfLEHmulti = pickle.load(f)
    return rfLEHmulti

## 3) read data

In [10]:
## 0) read era5land data
era5 = xr.open_zarr(ERA5_PATH)
# round coordinates to facilitate matching with other datasets
era5 = era5.assign_coords(
    longitude=era5.longitude.round(4),
    latitude=era5.latitude.round(4)
)

In [11]:
# 120 x 120 degrees
era5 = era5.isel(
    latitude=slice(0, 1250),  # (0., 125.) deg
    longitude=slice(0, 1250),  # (90., -35.) deg
)

Conversion to half-hourly quantities involves:

1. resampling to half-hourly grid;
2. differentiation with respect to time;

The order in which these tasks are carried matters: one should first resample the cumulative quantities to a finer grid and only then do the differentiation with respect to time. I think these tasks were carried out in the wrong order in an earlier version of the notebook.

In addition, the resampling to a half-hourly grid is problematic since it creates a single time chunk, increasing enormously the memory footprint of the calculation. Could we calculate half-hourly quantities (as I understood, these are required by the model) but on an hourly grid instead? As far as I understand, dividing by two `Rin` and `Rli` should bring them to half-hour-based unit. If results are required on a half-hourly grid, one could always interpolate results..   

In [12]:
# calculate the Rin and Rli difference for every hour
ssrd = era5['ssrd'] / 3600
Rin = ssrd.diff("time")
Rin[0::24] = ssrd[1::24]

strd = era5['strd'] / 3600
Rli = strd.diff("time")
Rli[0::24] = strd[1::24]

In [13]:
p = era5["sp"]/100  # Pa -> hPa
Ta = era5["t2m"] - 273.15  # K -> degC
ea = vc.calculate_es(era5["d2m"] - 273.15)
u = (era5["u10"] ** 2 + era5["v10"] ** 2) ** 0.5
Precip_msr = era5["tp"]*1000 # mm

In [14]:
### 1) read LAI data
LAI = xr.open_zarr(LAI_PATH)
LAI = LAI.assign_coords(
    longitude=LAI.longitude.round(4),
    latitude=LAI.latitude.round(4)
)

In [15]:
LAI = LAI['LAI']

In [16]:
LAI = LAI.isel(
    latitude=slice(0, 1250),  # (0., 125.) deg
    longitude=slice(0, 1250),  # (90., -35.) deg
)

Interpolation creates a single chunk in time! Can we use nearest-neighbour to match the time domain of the ERA5 dataset instead?

In [17]:
# INTERPOLATION CREATES A SINGLE CHUNK IN TIME!
# LAI = LAI.resample(time='1H').interpolate('linear')

The following Dask option allows to avoid creating a single time chunk when using nearest-neighbour matching: 

In [18]:
import dask.config

with dask.config.set({"array.slicing.split_large_chunks": True}):
    LAI = LAI.resample(time='1H').nearest()

**Done datasets up to here!** Moving on with landcover

----

In [ ]:
# ### 2) read canopy height data
# hc_path = inputData+year+'/canopy_height/canopy_height_11kmEurope20230921_10km.nc'
# hc = xr.open_dataset(hc_path,chunks={"y":110, "x":110}).rename({'x':'longitude','y':'latitude'})['__xarray_dataarray_variable__']

In [ ]:
# hc = hc.sel(
#     latitude=era5.latitude,  
#     longitude=era5.longitude,
#     method='nearest', tolerance=0.01
# )
# hc

In [ ]:
# ### 3) read CO2 data
# ## resampled CO2
# ds_co2_10km = xr.open_dataset(inputData+year+"/co2/CAMS_CO2_2015_10km.nc",chunks={"latitude":110, "longitude":110})['co2']
# ds_co2_10km = ds_co2_10km.sel(
#     latitude=era5.latitude,  
#     longitude=era5.longitude,
#     method='nearest', tolerance=0.01
# )

In [ ]:
# ds_co2_10km = ds_co2_10km.resample(time="1800S").interpolate('linear')

In [ ]:
# ### 4) read the resampled SSM data
# ## netcdf file works well
# path_SSM = glob.glob(inputData+year+"/ssm/SM2015Europe11kmEurope20230921_10km.nc")[0]
# ds_SSM = xr.open_dataset(path_SSM,chunks={"latitude":110, "longitude":110})['__xarray_dataarray_variable__'].rename('SSM')
# # ds_SSM = ds_SSM.sortby(["longitude", "latitude"])
# ds_SSM = ds_SSM.sel(
#     latitude=era5.latitude,  
#     longitude=era5.longitude,
#     method='nearest', tolerance=0.01
# )
# SSM = ds_SSM.resample(time="1800S").interpolate('linear')/1000 

In [ ]:
# ### 5) read Vcmax data
# ds_Vcmo = xr.open_dataset(inputData+year+"/vcmax/TROPOMI_Vmax_Tg_mean10km_global.nc",chunks={"y":110, "x":110}).rename({'x':'longitude','y':'latitude'})['__xarray_dataarray_variable__']

In [ ]:
# ds_Vcmo = ds_Vcmo.sel(
#     latitude=era5.latitude,  
#     longitude=era5.longitude,
#     method='nearest', tolerance=0.01
# )

---

In [20]:
## 6) read IGBP data
landcover = xr.open_zarr(LANDCOVER_PATH)
landcover = landcover.assign_coords(
    longitude=landcover.longitude.round(4),
    latitude=landcover.latitude.round(4)
)

In [21]:
landcover = landcover['lccs_class']

In [22]:
landcover = landcover.isel(
    latitude=slice(0, 1250),  # (0., 125.) deg
    longitude=slice(0, 1250),  # (90., -35.) deg
)

In [23]:
# read IGBP unique values
training_testing_append = pd.read_csv(IGBP_CLASS_PATH)['0'].unique()
# read the table for converting landcover to IGBP
IGBP_table = pd.read_csv(IGBP_TABLE_PATH)

In [24]:
def landcover_to_igbp(landcover, IGBP_table, training_testing_append):
    get_IGBP = np.vectorize(IGBP_table.set_index("lccs_class").T.to_dict('records')[0].get)
    IGBP = get_IGBP(landcover.values) 
    IGBP_all = pd.DataFrame(
        columns=[f'IGBP_veg_long{i}' for i in range(1, 12)]
    )
    
    # define one hot encoding for IGBP
    encoder = OneHotEncoder(
        categories=[training_testing_append],
        sparse=False,
        handle_unknown="ignore"
    )
    
    # transform data
    aa = encoder.fit_transform(IGBP.reshape(IGBP.shape[0]*IGBP.shape[1], 1))
    
    # assign 23-D IGBP into 23 columns
    for i in range(1, 12):
        IGBP_all[f'IGBP_veg_long{i}'] = aa[:,i-1]
    return IGBP_all

## 4) chunk all the input variables

**Missing data here!** I haven't loaded the CO2 and SSM datasets.

In [25]:
ds = xr.Dataset()

ds = ds.assign(
    Rin=Rin,
    Rli=Rli,
    p=p,
    Ta=Ta,
    ea=ea,
    u=u,
    Precip_msr=Precip_msr,
    LAI=LAI,
#######
    # CO2=CO2,
    # SSM=SSM,
#######
)

ds = ds.to_array()

ds = ds.chunk(time=125, variable=-1)

## 5) predict fluxes with map_blocks

In [26]:
INPUT_VARIABLES = [
    'Rin', 'Rli', 'p', 'Ta', 'ea', 'u', 'Precip_msr', 'LAI', 'hc', 'CO2',
    'SSM', 'Vcmo', *[f'IGBP_veg_long{i}' for i in range(1, 12)]
]
OUTPUT_VARIABLES = ['LEtot','Htot','Rntot', 'RSSM', 'SIF685', 'SIF740', 'Actot']

In [27]:
chunks = [ds.chunksizes[v] for v in ['time', 'latitude', 'longitude']]
chunks.append((len(OUTPUT_VARIABLES),))

template_LEH = xr.DataArray(
    name = 'LEH',
    data=da.zeros(
        (len(ds.time), len(ds.latitude), len(ds.longitude), len(OUTPUT_VARIABLES)), 
        chunks=chunks,
    ),
    dims=("time", "latitude", "longitude", "output_variable"),
    coords={
        "output_variable": OUTPUT_VARIABLES, 
        "time": ds.time, 
        "latitude": ds.latitude,
        "longitude": ds.longitude
    }
)

**Fake data here!** instead of taking `ds['CO2']` and `ds['SSM']` , I use `ds['p']` and `ds['Ta']`: 

In [28]:
def expand_time_dimension(data, n_time):
    """ 
    Expand the space-dependent data over the time dimension.
    
    Parameters
    ----------
    data : np.ndarray
        (ny, nx) matrix
    n_time : int
        number of elements in the time dimension
    
    Returns
    -------
    np.ndarray
        (1, ntime*ny*nx) matrix
    """
    expanded = np.tile(data.reshape(1, -1), (n_time, 1))
    return expanded.reshape(1, -1)
    

def predictFlux(ds, hc, Vcmo, landcover, IGBP_table, training_testing_append, path_model):

    n_time = len(ds.time)
    
    hc_ = expand_time_dimension(hc.data, n_time)
    Vcmo_ = expand_time_dimension(Vcmo.data, n_time)
    
    IGBP_all = landcover_to_igbp(landcover, IGBP_table, training_testing_append)
    IGBP_ = [
        expand_time_dimension(IGBP_all[f'IGBP_veg_long{i}'].to_numpy(), n_time)
        for i in range(1, 12)
    ]
    
    Rin_ = ds.sel(variable='Rin').data.reshape(1, -1)
    Rli_ = ds.sel(variable='Rli').data.reshape(1, -1)
    p_ = ds.sel(variable='p').data.reshape(1, -1)
    Ta_ = ds.sel(variable='Ta').data.reshape(1, -1)
    ea_ = ds.sel(variable='ea').data.reshape(1, -1)
    u_ = ds.sel(variable='u').data.reshape(1, -1)
    Precip_msr_ = ds.sel(variable='Precip_msr').data.reshape(1, -1)
    LAI_ = ds.sel(variable='LAI').data.reshape(1, -1)
###############
#    CO2_ = ds.sel(variable='CO2').data.reshape(1, -1)
#    SSM_ = ds.sel(variable='SSM').data.reshape(1, -1)
    CO2_ = ds.sel(variable='p').data.reshape(1, -1)
    SSM_ = ds.sel(variable='Ta').data.reshape(1, -1)
###############

    features_arr = np.concatenate((
        Rin_, Rli_, p_, Ta_, ea_, u_, Precip_msr_, LAI_, hc_, CO2_, SSM_, Vcmo_, *IGBP_
    ))
    features_arr = features_arr.transpose()
    df_features = pd.DataFrame(
        data=features_arr,
        columns=INPUT_VARIABLES,
    )
    invalid_index = df_features.isnull().any(axis=1)
    
    # Convert the nan value as 0 for the calculation
    df_features[invalid_index] = 0
    
    model = load_model(path_model)
    LEH = model.predict(df_features)
    LEH[invalid_index] = np.nan
    
    return xr.DataArray(
        name='LEH',
        data=LEH.reshape(len(ds.time), len(ds.latitude), len(ds.longitude), len(OUTPUT_VARIABLES)),
        dims=("time", "latitude", "longitude", "output_variable"),
        coords={
            "output_variable": OUTPUT_VARIABLES, 
            "time": ds.time, 
            "latitude": ds.latitude,
            "longitude":ds.longitude
        }
    )

**Fake data here!** instead of passing `hc, Vcmo, landcover`, I pass three times `landcover`: 

In [29]:
LEH = xr.map_blocks(
    predictFlux,
    ds,
#########
    # args=[hc, Vcmo, landcover],
    args=[landcover, landcover, landcover],
#########
    kwargs={
        "IGBP_table": IGBP_table, 
        "training_testing_append": training_testing_append, 
        "path_model": MODEL_PATH,
    },
    template=template_LEH,
)

In [30]:
LEH_ds = LEH.to_dataset(dim="output_variable") 

In [32]:
%%time
LEH_ds.to_zarr(LEH_PATH, mode='w')

/gpfs/home2/fnattino30/mambaforge/envs/emulator/lib/python3.9/site-packages/distributed/client.py:3149: UserWarning: Sending large graph of size 19.65 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


CPU times: user 3min 44s, sys: 10.1 s, total: 3min 54s
Wall time: 19min 26s


Release resources of the Dask cluster:

In [33]:
client.shutdown()

Verify the output is written:

In [34]:
! du -h $LEH_PATH

31G	/gpfs/work2/0/ttse0619/francesco/Projects/EcoExtreML/Data/5output_data/2015fluxes.zarr/Htot
2.0K	/gpfs/work2/0/ttse0619/francesco/Projects/EcoExtreML/Data/5output_data/2015fluxes.zarr/longitude
2.0K	/gpfs/work2/0/ttse0619/francesco/Projects/EcoExtreML/Data/5output_data/2015fluxes.zarr/latitude
24G	/gpfs/work2/0/ttse0619/francesco/Projects/EcoExtreML/Data/5output_data/2015fluxes.zarr/SIF685
2.0K	/gpfs/work2/0/ttse0619/francesco/Projects/EcoExtreML/Data/5output_data/2015fluxes.zarr/time
22G	/gpfs/work2/0/ttse0619/francesco/Projects/EcoExtreML/Data/5output_data/2015fluxes.zarr/Actot
31G	/gpfs/work2/0/ttse0619/francesco/Projects/EcoExtreML/Data/5output_data/2015fluxes.zarr/LEtot
24G	/gpfs/work2/0/ttse0619/francesco/Projects/EcoExtreML/Data/5output_data/2015fluxes.zarr/RSSM
35G	/gpfs/work2/0/ttse0619/francesco/Projects/EcoExtreML/Data/5output_data/2015fluxes.zarr/Rntot
24G	/gpfs/work2/0/ttse0619/francesco/Projects/EcoExtreML/Data/5output_data/2015fluxes.zarr/SIF740
188G	/gpfs/work2/0/tt

In [35]:
LEH = xr.open_zarr(LEH_PATH)

In [36]:
LEH

<xarray.Dataset>
Dimensions:    (time: 8760, latitude: 1250, longitude: 1250)
Coordinates:
  * latitude   (latitude) float64 90.0 89.9 89.8 89.7 ... -34.7 -34.8 -34.9
  * longitude  (longitude) float64 0.0 0.1 0.2 0.3 ... 124.6 124.7 124.8 124.9
  * time       (time) datetime64[ns] 2015-01-01 ... 2015-12-31T23:00:00
Data variables:
    Actot      (time, latitude, longitude) float64 dask.array<chunksize=(125, 250, 250), meta=np.ndarray>
    Htot       (time, latitude, longitude) float64 dask.array<chunksize=(125, 250, 250), meta=np.ndarray>
    LEtot      (time, latitude, longitude) float64 dask.array<chunksize=(125, 250, 250), meta=np.ndarray>
    RSSM       (time, latitude, longitude) float64 dask.array<chunksize=(125, 250, 250), meta=np.ndarray>
    Rntot      (time, latitude, longitude) float64 dask.array<chunksize=(125, 250, 250), meta=np.ndarray>
    SIF685     (time, latitude, longitude) float64 dask.array<chunksize=(125, 250, 250), meta=np.ndarray>
    SIF740     (time, latitude, longitude) float64 dask.array<chunksize=(125, 250, 250), meta=np.ndarray>